In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [27]:
# Hàm tính IOU
def compute_iou(box1, box2):
    """ Tính Intersection over Union (IOU) giữa hai bounding boxes. """
    x1 = np.maximum(box1[0], box2[0])
    y1 = np.maximum(box1[1], box2[1])
    x2 = np.minimum(box1[2], box2[2])
    y2 = np.minimum(box1[3], box2[3])
    
    intersection_area = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    union_area = box1_area + box2_area - intersection_area
    
    iou = intersection_area / union_area
    return iou


In [28]:
# Tạo mô hình Faster R-CNN đơn giản với RPN và bounding box regression
def create_faster_rcnn_model():
    input_layer = Input(shape=(None, None, 3))
    
    # RPN (Region Proposal Network)
    rpn_conv = Conv2D(256, (3, 3), activation='relu', padding='same')(input_layer)
    rpn_cls = Conv2D(9, (1, 1), activation='sigmoid', name='rpn_cls')(rpn_conv)  # 9 là số lượng anchor boxes
    rpn_reg = Conv2D(36, (1, 1), activation='linear', name='rpn_reg')(rpn_conv)  # 36 = 4 (bounding box deltas) * 9
    
    # Flatten output của RPN
    rpn_cls_flat = Flatten()(rpn_cls)
    rpn_reg_flat = Flatten()(rpn_reg)
    
    # Tạo mô hình
    rpn_model = Model(inputs=input_layer, outputs=[rpn_cls_flat, rpn_reg_flat])
    return rpn_model


In [29]:
# Chuẩn bị dữ liệu
train_dir = 'O:/Rcnn_Plant_diseases/Dataset/train'
valid_dir = 'O:/Rcnn_Plant_diseases/Dataset/valid'

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [30]:
# Xây dựng mô hình Faster R-CNN
rpn_model = create_faster_rcnn_model()


In [31]:
# Tạo input và output placeholders cho classification và bounding box regression
input_layer = Input(shape=(None, None, 3))
rpn_cls_pred, rpn_reg_pred = rpn_model(input_layer)

In [32]:
# Tạo mô hình chính với input là input_layer và output là rpn_cls_pred và rpn_reg_pred
rcnn_model = Model(inputs=input_layer, outputs=[rpn_cls_pred, rpn_reg_pred])

In [33]:
# Compile mô hình với optimizer và loss function
rcnn_model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss=[SparseCategoricalCrossentropy(), tf.keras.losses.MeanSquaredError()],
                   metrics=['accuracy', tf.keras.metrics.MeanSquaredError()])

# Compile mô hình với optimizer và loss function
# rcnn_model.compile(optimizer=Adam(learning_rate=0.0001),
#                    loss=[SparseCategoricalCrossentropy(), mse],
#                    metrics=['accuracy', mse])



In [36]:
# Huấn luyện mô hình
rcnn_history = rcnn_model.fit(train_generator, epochs=10, validation_data=valid_generator)


Epoch 1/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 2014s 916ms/step - functional_9_accuracy: 0.0172 - loss: 9.3199 - val_functional_9_accuracy: 0.0214 - val_loss: 7.0651
Epoch 2/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 2009s 914ms/step - functional_9_accuracy: 0.0205 - loss: 7.0243 - val_functional_9_accuracy: 0.0219 - val_loss: 6.9654
Epoch 3/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1982s 902ms/step - functional_9_accuracy: 0.0208 - loss: 6.9598 - val_functional_9_accuracy: 0.0266 - val_loss: 6.9230
Epoch 4/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1972s 897ms/step - functional_9_accuracy: 0.0229 - loss: 6.9351 - val_functional_9_accuracy: 0.0273 - val_loss: 6.8933
Epoch 5/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1918s 873ms/step - functional_9_accuracy: 0.0244 - loss: 6.8902 - val_functional_9_accuracy: 0.0277 - val_loss: 6.8707
Epoch 6/10
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1849s 841ms/step - functional_9_accuracy: 0.0329 - loss: 6.8759 - val_functional_9_accuracy: 0.0418 - val_loss: 6.8537
Epoch 7/10
2197/2197 ━━━━━━━━━━━━━

In [37]:
# Lưu mô hình
rcnn_model.save('faster_rcnn_new_model.h5')

In [38]:
# Lịch sử huấn luyện được trả về từ phương thức fit()
history = rcnn_history.history

# Lưu lịch sử huấn luyện vào một tệp JSON
import json
with open('training_new_history.json', 'w') as f:
    json.dump(history, f)
